In [13]:
import pandas as pd
from pathlib import Path

# ==== 1) Okunacak Parquet dosyasını belirtin ====
parquet_file = Path("data/tefas_full_2yrs.parquet")   # burayı değiştirin

# ===============================================

# Parquet dosyasını oku
if not parquet_file.exists():
    raise FileNotFoundError(f"Parquet bulunamadı: {parquet_file}")

df = pd.read_parquet(parquet_file)
display(df.head())                 # ilk satırları göster
print(f"Satır: {len(df):,}  |  Sütun: {df.shape[1]}")

# ==== 2) Excel’e dönüştür ====
excel_file = parquet_file.with_suffix(".xlsx")
df.to_excel(excel_file, index=False)
print(f"Excel dosyası oluşturuldu: {excel_file}")

,tarih,fiyat,tedavuldeki_pay_sayisi,toplam_deger,yatirimci_sayisi,fon_unvan,fon_kodu,fon_kategorisi
0,2023-07-14,12.85490,6881972.0,88467064.59,535,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
1,2023-07-17,12.94562,6822709.0,88324200.73,540,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
2,2023-07-18,13.16363,6844889.0,90103583.75,545,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
3,2023-07-19,13.13001,6616862.0,86879466.07,554,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
4,2023-07-20,13.39776,6663617.0,89277539.59,556,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu


Satır: 337,737  |  Sütun: 8
Excel dosyası oluşturuldu: data\tefas_full_2yrs.xlsx


In [4]:
# ==== 3) Fon Kategorilerini Tekrar Hesapla ====
# Bu bölüm tefas_download_data_merged.py'deki güncellenmiş kategori mantığını kullanarak
# mevcut parquet dosyasındaki fon_kategorisi sütununu yeniden hesaplar
import pandas as pd

parquet_file = Path("data/tefas_full_2yrs.parquet")   # burayı değiştirin

def normalize_turkish_text(text):
    """Türkçe karakter normalizasyonu"""
    if not text:
        return ""
    replacements = {
        'Ğ': 'G', 'ğ': 'g', 'Ü': 'U', 'ü': 'u', 'Ş': 'S', 'ş': 's',
        'İ': 'I', 'ı': 'i', 'Ö': 'O', 'ö': 'o', 'Ç': 'C', 'ç': 'c'
    }
    for tr, en in replacements.items():
        text = text.replace(tr, en)
    return text.upper()

def guess_category_from_name(fund_name):
    """Fon adından kategori tahmini - priorite sıralı kurallar"""
    if not fund_name:
        return "Bilinmeyen Şemsiye Fonu"
    
    name_normalized = normalize_turkish_text(fund_name)
    
    # ÖNCELIK SIRALI KURALLAR (Kullanıcı gereksinimleri)
    # 1. SERBEST geçiyorsa kesinlikle serbest (SERBEST + DEĞİŞKEN veya SERBEST + KATILIM olsa bile)
    if any(keyword in name_normalized for keyword in ["SERBEST", "FLEXIBLE"]):
        return "Serbest Şemsiye Fonu"
    
    # 2. DEĞİŞKEN geçiyorsa kesinlikle değişken (DEĞİŞKEN + KATILIM olsa bile)
    elif any(keyword in name_normalized for keyword in ["DEGISKEN", "VARIABLE"]):
        return "Değişken Şemsiye Fonu"
    
    # 3. KATILIM geçiyorsa kesinlikle katılım (KATILIM + HİSSE olsa bile)
    elif any(keyword in name_normalized for keyword in ["KATILIM", "PARTICIPATION", "SUKUK"]):
        return "Katılım Şemsiye Fonu"
    
    # DİĞER KATEGORI TANIMLARı (mevcut mantık)
    # 4. YABANCI + HİSSE kombinasyonu (hem YABANCI hem HİSSE geçmeli)
    elif all(keyword in name_normalized for keyword in ["YABANCI", "HISSE"]):
        return "Yabancı Hisse Senedi Şemsiye Fonu"
    elif any(keyword in name_normalized for keyword in ["HISSE", "EQUITY", "STOCK"]):
        return "Hisse Senedi Şemsiye Fonu"
    elif any(keyword in name_normalized for keyword in ["PARA PIYASASI", "MONEY MARKET"]):
        return "Para Piyasası Şemsiye Fonu"
    elif any(keyword in name_normalized for keyword in ["FON SEPETI", "FUND BASKET"]):
        return "Fon Sepeti Şemsiye Fonu"
    elif any(keyword in name_normalized for keyword in ["ALTIN", "GOLD", "GUMUSH", "SILVER", "KIYMETLI"]):
        return "Kıymetli Madenler Şemsiye Fonu"
    elif any(keyword in name_normalized for keyword in ["EURO", "EUROBOND"]):
        return "Eurobond Şemsiye Fonu"
    elif any(keyword in name_normalized for keyword in ["BORCLANMA", "BOND", "TAHVIL"]):
        return "Borçlanma Araçları Şemsiye Fonu"
    else:
        return "Diğer Şemsiye Fonu"

# ================================================================
# Kategori güncelleme işlemi
# ================================================================

print("=== FON KATEGORİLERİNİ YENİDEN HESAPLAMA ===")
print(f"Parquet dosyası: {parquet_file}")

# Mevcut verileri kopyala
df_updated = df.copy()

# Mevcut kategori dağılımını göster
print("\n=== MEVCUT KATEGORI DAĞILIMI ===")
current_categories = df_updated['fon_kategorisi'].value_counts()
print(current_categories)

# Yeni kategorileri hesapla
print("\n=== YENİ KATEGORİLER HESAPLANIYOR ===")
df_updated['fon_kategorisi_yeni'] = df_updated['fon_unvan'].apply(guess_category_from_name)

# Değişiklikleri analiz et
changes = df_updated[df_updated['fon_kategorisi'] != df_updated['fon_kategorisi_yeni']]
print(f"\nDeğişen fon sayısı: {len(changes)} / {len(df_updated)}")

if len(changes) > 0:
    print("\n=== DEĞİŞEN KATEGORILER ===")
    change_cols = changes[['fon_kodu', 'fon_unvan', 'fon_kategorisi', 'fon_kategorisi_yeni']]
    change_summary = change_cols.drop_duplicates(subset='fon_kodu')
    
    for _, row in change_summary.iterrows():
        print(f"• {row['fon_kodu']}: {row['fon_kategorisi']} → {row['fon_kategorisi_yeni']}")
        print(f"  Fon adı: {row['fon_unvan']}")
        print()

# Eski kategorileri yeni kategorielerle değiştir
df_updated['fon_kategorisi'] = df_updated['fon_kategorisi_yeni']
df_updated.drop('fon_kategorisi_yeni', axis=1, inplace=True)

# Yeni kategori dağılımını göster
print("\n=== YENİ KATEGORI DAĞILIMI ===")
new_categories = df_updated['fon_kategorisi'].value_counts()
print(new_categories)

# Özet bilgiler
unique_changed_funds = changes.drop_duplicates(subset='fon_kodu')
print(f"\n=== ÖZET ===")
print(f"Toplam fon sayısı: {df_updated['fon_kodu'].nunique()}")
print(f"Toplam kayıt sayısı: {len(df_updated)}")
print(f"Kategori değişen fon sayısı: {len(unique_changed_funds)}")


=== FON KATEGORİLERİNİ YENİDEN HESAPLAMA ===
Parquet dosyası: data\tefas_full_2yrs.parquet

=== MEVCUT KATEGORI DAĞILIMI ===
fon_kategorisi
Serbest Şemsiye Fonu                 103861
Değişken Şemsiye Fonu                 57636
Hisse Senedi Şemsiye Fonu             48455
Fon Sepeti Şemsiye Fonu               34594
Katılım Şemsiye Fonu                  33446
Borçlanma Araçları Şemsiye Fonu       20539
Para Piyasası Şemsiye Fonu            16922
Kıymetli Madenler Şemsiye Fonu         7571
Yabancı Hisse Senedi Şemsiye Fonu      5508
Diğer Şemsiye Fonu                     5200
Eurobond Şemsiye Fonu                  4005
Name: count, dtype: int64

=== YENİ KATEGORİLER HESAPLANIYOR ===

Değişen fon sayısı: 1686 / 337737

=== DEĞİŞEN KATEGORILER ===
• AAK: Diğer Şemsiye Fonu → Değişken Şemsiye Fonu
  Fon adı: ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON

• AAL: Diğer Şemsiye Fonu → Para Piyasası Şemsiye Fonu
  Fon adı: ATA PORTFÖY PARA PİYASASI (TL) FONU

• AAS: Diğer Şemsiye Fonu → Serbest Şemsiye 

In [8]:
# ==== 4) Güncellenmiş Veriyi Kaydet ====

# Parquet dosyasını güncelle (orijinalin yedeğini al)
backup_file = parquet_file.with_suffix(f".backup_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.parquet")
print(f"\n=== KAYDETME İŞLEMİ ===")
print(f"Orijinal dosya yedeği: {backup_file}")

# Orijinalin yedeğini al
df.to_parquet(backup_file)
print(f"✅ Yedek dosya oluşturuldu")

# Güncellenmiş veriyi kaydet
df_updated.to_parquet(parquet_file)
print(f"✅ Güncellenmiş veri kaydedildi: {parquet_file}")

# Karşılaştırma Excel'i oluştur (sadece değişen fonlar)
if len(changes) > 0:
    comparison_file = parquet_file.with_name(f"kategori_karsilastirma_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
    
    # Değişen fonların özet tablosunu oluştur
    change_cols = changes[['fon_kodu', 'fon_unvan', 'fon_kategorisi', 'fon_kategorisi_yeni']]
    unique_changes = change_cols.drop_duplicates(subset='fon_kodu')
    unique_changes.rename(columns={
        'fon_kategorisi': 'ESKİ_KATEGORİ', 
        'fon_kategorisi_yeni': 'YENİ_KATEGORİ'
    }, inplace=True)
    
    # Excel'e yaz
    with pd.ExcelWriter(comparison_file, engine='openpyxl') as writer:
        unique_changes.to_excel(writer, sheet_name='Değişen_Kategoriler', index=False)
        
        # Kategori dağılım karşılaştırması
        comparison_stats = pd.DataFrame({
            'ESKİ_DAĞILIM': current_categories,
            'YENİ_DAĞILIM': new_categories
        }).fillna(0)
        comparison_stats.to_excel(writer, sheet_name='Kategori_Dağılımı')
        
    print(f"✅ Karşılaştırma Excel'i oluşturuldu: {comparison_file}")
else:
    print("ℹ️  Kategori değişikliği olmadığı için karşılaştırma dosyası oluşturulmadı")

print("\n=== İŞLEM TAMAMLANDI ===")
print(f"📁 Ana dosya: {parquet_file}")
print(f"📁 Yedek dosya: {backup_file}")
if len(changes) > 0:
    print(f"📁 Karşılaştırma: {comparison_file}")

# Son kontrol
df_final = pd.read_parquet(parquet_file)
print(f"\n✅ Doğrulama: Kaydedilen dosyada {df_final['fon_kodu'].nunique()} fon, {len(df_final)} kayıt bulunuyor")



=== KAYDETME İŞLEMİ ===
Orijinal dosya yedeği: data\tefas_full_2yrs.backup_20250716_123629.parquet
✅ Yedek dosya oluşturuldu
✅ Güncellenmiş veri kaydedildi: data\tefas_full_2yrs.parquet
✅ Karşılaştırma Excel'i oluşturuldu: data\kategori_karsilastirma_20250716_123629.xlsx

=== İŞLEM TAMAMLANDI ===
📁 Ana dosya: data\tefas_full_2yrs.parquet
📁 Yedek dosya: data\tefas_full_2yrs.backup_20250716_123629.parquet
📁 Karşılaştırma: data\kategori_karsilastirma_20250716_123629.xlsx

✅ Doğrulama: Kaydedilen dosyada 850 fon, 336037 kayıt bulunuyor


C:\Users\PC\AppData\Local\Temp\ipykernel_30272\1853492413.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_changes.rename(columns={


In [1]:
# =======================================================================
# FON KATEGORİLERİ KONTROL VE EXCEL EXPORT
# =======================================================================

import pandas as pd
from pathlib import Path

# Dosya yolu
parquet_file = Path("data/tefas_full_2yrs.parquet")

# Parquet dosyasını oku
if not parquet_file.exists():
    print(f"❌ Parquet bulunamadı: {parquet_file}")
    print("📁 Mevcut data klasöründeki dosyalar:")
    data_dir = Path("data")
    if data_dir.exists():
        for file in data_dir.glob("*.parquet"):
            print(f"   - {file.name}")
    else:
        print("   Data klasörü bulunamadı!")
else:
    print(f"✅ Dosya bulundu: {parquet_file}")
    
    # Veriyi oku
    df = pd.read_parquet(parquet_file)
    print(f"📊 Toplam kayıt: {len(df):,}")
    print(f"📊 Sütun sayısı: {df.shape[1]}")
    print(f"📊 Sütunlar: {list(df.columns)}")
    
    # Temel bilgiler
    if 'fon_kodu' in df.columns:
        unique_funds = df['fon_kodu'].nunique()
        print(f"📊 Unique fon sayısı: {unique_funds}")
    
    # Tarih aralığı
    if 'tarih' in df.columns:
        df['tarih'] = pd.to_datetime(df['tarih'])
        min_date = df['tarih'].min().strftime('%Y-%m-%d')
        max_date = df['tarih'].max().strftime('%Y-%m-%d')
        print(f"📅 Tarih aralığı: {min_date} ~ {max_date}")
    
    print("\n" + "="*50)
    print("🔍 FON DETAY ANALİZİ")
    print("="*50)


✅ Dosya bulundu: data\tefas_full_2yrs.parquet
📊 Toplam kayıt: 337,737
📊 Sütun sayısı: 8
📊 Sütunlar: ['tarih', 'fiyat', 'tedavuldeki_pay_sayisi', 'toplam_deger', 'yatirimci_sayisi', 'fon_unvan', 'fon_kodu', 'fon_kategorisi']
📊 Unique fon sayısı: 850
📅 Tarih aralığı: 2023-07-14 ~ 2025-07-17

🔍 FON DETAY ANALİZİ


In [2]:
# =======================================================================
# UNIQUE FON BİLGİLERİNİ ÇIKAR VE EXCEL'E BAS
# =======================================================================

# Gerekli sütunları kontrol et
required_cols = ['fon_kodu', 'fon_unvan', 'fon_kategorisi']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"❌ Eksik sütunlar: {missing_cols}")
    print(f"📊 Mevcut sütunlar: {list(df.columns)}")
else:
    print("✅ Gerekli sütunlar mevcut!")
    
    # Unique fon bilgilerini al
    # Her fon için ilk kaydı alıyoruz (çünkü fon bilgileri aynı olmalı)
    unique_funds_df = df.groupby('fon_kodu').first().reset_index()[required_cols]
    
    print(f"📊 Unique fon sayısı: {len(unique_funds_df)}")
    
    # Kategori dağılımını göster
    print("\n🏷️ FON KATEGORİ DAĞILIMI:")
    category_series = unique_funds_df['fon_kategorisi']
    category_counts = category_series.value_counts()
    for category, count in category_counts.items():
        percentage = (count / len(unique_funds_df)) * 100
        print(f"   📈 {category}: {count:3d} fon ({percentage:5.1f}%)")
    
    # İlk 10 fonu göster
    print(f"\n📋 İLK 10 FON ÖRNEĞİ:")
    print(unique_funds_df.head(10).to_string(index=False))
    
    # Excel'e kaydet
    excel_file = Path("data/fon_kategorileri_kontrol.xlsx")
    unique_funds_df.to_excel(excel_file, index=False, sheet_name="Fon_Kategorileri")
    
    print(f"\n✅ Excel dosyası oluşturuldu: {excel_file}")
    print(f"📊 Toplam satır: {len(unique_funds_df)}")
    print(f"📊 Sütunlar: {list(unique_funds_df.columns)}")
    
    # Potansiyel problemleri kontrol et
    print(f"\n🔍 KALİTE KONTROLLERI:")
    
    # Boş kategori kontrolü
    empty_categories = category_series.isna().sum()
    print(f"   📋 Boş kategori: {empty_categories} fon")
    
    # 'Diğer' kategorisi kontrolü
    other_categories = category_series.str.contains('Diğer', case=False, na=False).sum()
    print(f"   📋 'Diğer' kategorisi: {other_categories} fon")
    
    # Çok kısa fon adları (potansiyel veri problemi)
    name_series = unique_funds_df['fon_unvan']
    short_names = (name_series.str.len() < 10).sum()
    print(f"   📋 Çok kısa fon adı (<10 karakter): {short_names} fon")
    
    if empty_categories > 0 or other_categories > 50 or short_names > 0:
        print(f"\n⚠️  Potansiyel veri kalite problemleri tespit edildi!")
    else:
        print(f"\n✅ Veri kalitesi iyi görünüyor!")


✅ Gerekli sütunlar mevcut!
📊 Unique fon sayısı: 850

🏷️ FON KATEGORİ DAĞILIMI:
   📈 Serbest Şemsiye Fonu: 281 fon ( 33.1%)
   📈 Değişken Şemsiye Fonu: 136 fon ( 16.0%)
   📈 Hisse Senedi Şemsiye Fonu: 128 fon ( 15.1%)
   📈 Katılım Şemsiye Fonu:  93 fon ( 10.9%)
   📈 Fon Sepeti Şemsiye Fonu:  73 fon (  8.6%)
   📈 Borçlanma Araçları Şemsiye Fonu:  50 fon (  5.9%)
   📈 Para Piyasası Şemsiye Fonu:  43 fon (  5.1%)
   📈 Kıymetli Madenler Şemsiye Fonu:  20 fon (  2.4%)
   📈 Yabancı Hisse Senedi Şemsiye Fonu:  11 fon (  1.3%)
   📈 Eurobond Şemsiye Fonu:   8 fon (  0.9%)
   📈 Diğer Şemsiye Fonu:   7 fon (  0.8%)

📋 İLK 10 FON ÖRNEĞİ:
fon_kodu                                                                                    fon_unvan             fon_kategorisi
     AAK                                                        ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON      Değişken Şemsiye Fonu
     AAL                                                          ATA PORTFÖY PARA PİYASASI (TL) FONU Para Pi